In [ ]:
#importing libraries
import pandas as pd
import nltk 
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import string
import nltk
import warnings 
import re 

In [ ]:
#Reading the trainig data
data = pd.read_csv('C:\\Users\\warnak\\OneDrive - John Keells Holdings PLC\\workspace\\msc\\ML\\train.txt', sep='\t')

In [ ]:
#reading the test data
test = pd.read_csv('C:\\Users\\warnak\\OneDrive - John Keells Holdings PLC\\workspace\\msc\\ML\\test.csv')

# Pre-Processing steps

In [ ]:
#combining the 2 datasets for preprocessing 
combi = data.append(test, ignore_index=True)

In [ ]:
#function to remove a given pattern in a string
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt

In [ ]:
#Removing mentions in tweets
combi['tidy_tweet'] = np.vectorize(remove_pattern)(combi['Tweet text'], "@[\w]*")

In [ ]:
#Removing special characters, numbers, punctuations
combi['tidy_tweet'] = combi['tidy_tweet'].str.replace("[^a-zA-Z#]", " ")

In [ ]:
#Removing the which have less than 3 characters 
combi['tidy_tweet'] = combi['tidy_tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

In [ ]:
#tokenizing the tweets 
tokenized_tweet = combi['tidy_tweet'].apply(lambda x: x.split())
tokenized_tweet.head()

In [ ]:
#adding the tokenized tweets 
for i in range(len(tokenized_tweet)):
    tokenized_tweet[i] = ' '.join(tokenized_tweet[i])

combi['tidy_tweet'] = tokenized_tweet

In [ ]:
#creating a bag of words
from sklearn.feature_extraction.text import CountVectorizer
bow_vectorizer = CountVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')
# bag-of-words feature matrix
bow = bow_vectorizer.fit_transform(combi['tidy_tweet'])

# Model Building

In [ ]:
#SVM model using BOW as feature 
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

train_bow = bow[:3817,:]
test_bow = bow[3817:,:]

# splitting data into training and validation set
xtrain_bow, xvalid_bow, ytrain, yvalid = train_test_split(train_bow, data['Label'], random_state=42, test_size=0.3)

clf = SVC(gamma='auto', probability=True)
clf.fit(xtrain_bow, ytrain) # training the model

prediction = clf.predict_proba(xvalid_bow) # predicting on the validation set
prediction_int = prediction[:,1] >= 0.3 # if prediction is greater than or equal to 0.3 than 1 else 0
prediction_int = prediction_int.astype(np.int)

f1_score(yvalid, prediction_int) # calculating f1 score

# Getting the predictions

In [ ]:
test_pred = clf.predict_proba(test_bow)
test_pred_int = test_pred[:,1] >= 0.3
test_pred_int = test_pred_int.astype(np.int)
test['Label'] = test_pred_int

In [ ]:
submission = test[['Tweet index','Label']]
submission.to_csv('sub_svm_bow.csv', index=False) # writing data to a CSV file